## import

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import sqlite3

## スクレイピング → repo_data

In [3]:
# Google 組織のリポジトリ一覧ページからリンクを取得
url = "https://github.com/orgs/google/repositories"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

repos = soup.select('h3 a[href^="/google/"]')
print("取得したリポジトリ数:", len(repos))

repo_data = []

for repo in repos[:10]:  # 例として先頭10件
    name = repo.get_text(strip=True)
    href = repo["href"]
    repo_url = "https://github.com" + href

    print("\n--- アクセス中:", name, repo_url)

    r = requests.get(repo_url, headers=headers)
    s = BeautifulSoup(r.text, "html.parser")

    # 主要言語
    lang_tag = s.find("span", {"itemprop": "programmingLanguage"})
    if lang_tag:
        lang = lang_tag.get_text(strip=True)
    else:
        lang = "Unknown"

    # スター数
    star_link = s.find("a", href=lambda h: h and h.endswith("/stargazers"))
    if star_link:
        stars_text = star_link.get_text(strip=True)
        stars_text = stars_text.replace(",", "")
        stars_text = stars_text.replace("stars", "")
        stars_text = stars_text.strip()

        if stars_text.lower().endswith("k"):
            num = stars_text.lower().replace("k", "")
            stars = int(float(num) * 1000)
        else:
            try:
                stars = int(float(stars_text))
            except ValueError:
                stars = 0
    else:
        stars = 0

    print("  主要言語:", lang)
    print("  スター数:", stars)

    repo_data.append((name, lang, stars))

    time.sleep(1)  # 1秒スリープ

print("\n収集したデータ件数:", len(repo_data))
print("先頭3件の例:", repo_data[:3])

取得したリポジトリ数: 30

--- アクセス中: sedpack https://github.com/google/sedpack
  主要言語: Unknown
  スター数: 28

--- アクセス中: tunix https://github.com/google/tunix
  主要言語: Unknown
  スター数: 1900

--- アクセス中: go-containerregistry https://github.com/google/go-containerregistry
  主要言語: Unknown
  スター数: 3600

--- アクセス中: bazel-common https://github.com/google/bazel-common
  主要言語: Unknown
  スター数: 91

--- アクセス中: XNNPACK https://github.com/google/XNNPACK
  主要言語: Unknown
  スター数: 2200

--- アクセス中: osv-scalibr https://github.com/google/osv-scalibr
  主要言語: Unknown
  スター数: 536

--- アクセス中: open-dice https://github.com/google/open-dice
  主要言語: Unknown
  スター数: 26

--- アクセス中: heir https://github.com/google/heir
  主要言語: Unknown
  スター数: 609

--- アクセス中: dive https://github.com/google/dive
  主要言語: Unknown
  スター数: 17

--- アクセス中: chromium-policy-vulnfeed https://github.com/google/chromium-policy-vulnfeed
  主要言語: Unknown
  スター数: 7

収集したデータ件数: 10
先頭3件の例: [('sedpack', 'Unknown', 28), ('tunix', 'Unknown', 1900), ('go-containerregistry

## SQLite に保存して SELECT 表示

In [4]:
path = ''
db_name = 'github_google.db'

try:
    conn = sqlite3.connect(path + db_name)
    cur = conn.cursor()

    # テーブル作成
    sql = """
    CREATE TABLE IF NOT EXISTS repos (
        id    INTEGER PRIMARY KEY AUTOINCREMENT,
        name  TEXT,
        lang  TEXT,
        stars INTEGER
    );
    """
    cur.execute(sql)
    conn.commit()

    # データ挿入
    insert_sql = "INSERT INTO repos (name, lang, stars) VALUES (?, ?, ?);"
    cur.executemany(insert_sql, repo_data)
    conn.commit()

    # SELECT で表示
    print("=== DB に保存されたデータ ===")
    select_sql = "SELECT id, name, lang, stars FROM repos;"
    cur.execute(select_sql)

    for row in cur:
        print(row)

except sqlite3.Error as e:
    print("エラーが発生しました:", e)

finally:
    conn.close()

=== DB に保存されたデータ ===
(1, 'site-kit-wp', 'Unknown', 1300)
(2, 'oss-fuzz', 'Unknown', 11600)
(3, 'adk-samples', 'Unknown', 6400)
(4, 'yggdrasil-decision-forests', 'Unknown', 620)
(5, 'googletest-rust', 'Unknown', 391)
(6, 'osv-scalibr', 'Unknown', 532)
(7, 'perfetto', 'Unknown', 5000)
(8, 'device-infra', 'Unknown', 58)
(9, 'orbax', 'Unknown', 452)
(10, 'testrun', 'Unknown', 42)
(11, 'sedpack', 'Unknown', 28)
(12, 'tunix', 'Unknown', 1900)
(13, 'go-containerregistry', 'Unknown', 3600)
(14, 'bazel-common', 'Unknown', 91)
(15, 'XNNPACK', 'Unknown', 2200)
(16, 'osv-scalibr', 'Unknown', 536)
(17, 'open-dice', 'Unknown', 26)
(18, 'heir', 'Unknown', 609)
(19, 'dive', 'Unknown', 17)
(20, 'chromium-policy-vulnfeed', 'Unknown', 7)
